# DSSM

<img src="../img/dssm.png" width="800">

In [1]:
import pandas as pd
import numpy as np
import json
import torch
from torch import nn
from torch.utils.data.dataset import Dataset
from collections import OrderedDict

In [2]:
learning_rate = 0.0001
batch_size = 64
num_epochs = 5
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [3]:
with open('../data/movielens/feature_map.json') as obj:
    feature_map = json.load(obj)
feature_map

{'dataset_id': 'movielens',
 'num_fields': 26,
 'feature_specs': {'movieId': {'source': 'item',
   'type': 'categorical',
   'vocab_size': 935,
   'index': 0},
  'userId': {'source': 'user',
   'type': 'categorical',
   'vocab_size': 22540,
   'index': 1},
  'rating': {'source': 'user',
   'type': 'numerical',
   'vocab_size': 1,
   'index': 2},
  'timestamp': {'source': 'user',
   'type': 'numerical',
   'vocab_size': 1,
   'index': 3},
  'releaseYear': {'source': 'item',
   'type': 'numerical',
   'vocab_size': 1,
   'index': 5},
  'movieGenre1': {'source': 'item',
   'type': 'categorical',
   'vocab_size': 18,
   'index': 6},
  'movieGenre2': {'source': 'item',
   'type': 'categorical',
   'vocab_size': 18,
   'index': 7},
  'movieGenre3': {'source': 'item',
   'type': 'categorical',
   'vocab_size': 15,
   'index': 8},
  'movieRatingCount': {'source': 'item',
   'type': 'numerical',
   'vocab_size': 1,
   'index': 9},
  'movieAvgRating': {'source': 'item',
   'type': 'numerical',
 

In [4]:
class MovielensDataset(Dataset):
    def __init__(self, url):
        self.df = pd.read_csv(url)
    
    def __getitem__(self, idx):
        x, y = self.df.iloc[idx, :-1].values.astype(np.float32), self.df.iloc[idx, -1].astype(np.float32)
        return x, y
    
    def __len__(self):
        return self.df.shape[0]

In [5]:
train_dataset = MovielensDataset('../data/movielens/data_for_train.csv')
test_dataset = MovielensDataset('../data/movielens/data_for_test.csv')

In [6]:
# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)

In [7]:
class DSSM(nn.Module):
    def __init__(self,
                 feature_map,
                 embedding_dim=10,
                 user_tower_units=[256, 128, 64],
                 item_tower_units=[256, 128, 64]):
        super(DSSM, self).__init__()
        self.feature_map = feature_map
        self.embedding = nn.ModuleDict()
        for feature, feature_spec in feature_map['feature_specs'].items():
            if feature_spec['type'] == 'numerical':
                self.embedding[feature] = nn.Linear(1, embedding_dim, bias=False)
            elif feature_spec['type'] == 'categorical':
                padding_idx = feature_spec.get('padding_idx', None)
                self.embedding[feature] = nn.Embedding(feature_spec['vocab_size'],
                                                       embedding_dim,
                                                       padding_idx=padding_idx)
        user_fields = sum(1 if feature_spec['source'] == 'user' else 0
                          for _, feature_spec in feature_map['feature_specs'].items())
        item_fields = sum(1 if feature_spec['source'] == 'item' else 0
                          for _, feature_spec in feature_map['feature_specs'].items())
        user_tower_units = [embedding_dim * user_fields] + user_tower_units
        item_tower_units = [embedding_dim * item_fields] + item_tower_units
        user_layers = []
        item_layers = []
        for i in range(len(user_tower_units) - 1):
            user_layers.append(nn.Linear(user_tower_units[i], user_tower_units[i + 1]))
            user_layers.append(nn.ReLU())
            item_layers.append(nn.Linear(item_tower_units[i], item_tower_units[i + 1]))
            item_layers.append(nn.ReLU())
        user_layers.append(nn.Linear(user_tower_units[-1], 32))
        item_layers.append(nn.Linear(item_tower_units[-1], 32))
        self.user_tower = nn.Sequential(*user_layers)
        self.item_tower = nn.Sequential(*item_layers)
        self.output_activation = nn.Sigmoid()

    def forward(self, X):
        feature_emb_dict = OrderedDict()
        for feature, feature_spec in self.feature_map['feature_specs'].items():
            if feature_spec['type'] == 'numerical':
                raw_feature = X[:, feature_spec['index']].float().view(-1, 1)
            elif feature_spec['type'] == 'categorical':
                raw_feature = X[:, feature_spec['index']].long()
            embedding_vec = self.embedding[feature](raw_feature)
            feature_emb_dict[feature] = embedding_vec
        # transfer to tensor
        user_emb_list = []
        item_emb_list = []
        for feature, feature_spec in self.feature_map['feature_specs'].items():
            if feature_spec['source'] == 'user':
                user_emb_list.append(feature_emb_dict[feature])
            else:
                item_emb_list.append(feature_emb_dict[feature])
        user_emb = torch.stack(user_emb_list, dim=1)
        item_emb = torch.stack(item_emb_list, dim=1)
        user_out = self.user_tower(user_emb.flatten(start_dim=1))
        item_out = self.item_tower(item_emb.flatten(start_dim=1))
        y_pred = self.output_activation((user_out * item_out).sum(dim=-1)).float()
        return y_pred

In [8]:
model = DSSM(feature_map).to(device)
# Loss and optimizer
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (X, y) in enumerate(train_loader):
        X = X.to(device)
        y = y.to(device)

        # Forward pass
        output = model(X)
        loss = criterion(output, y)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i + 1) % 300 == 0:
            print("Epoch [{}/{}], Step [{}/{}] Loss: {:.4f}".format(
                epoch + 1, num_epochs, i + 1, total_step, loss.item()))

Epoch [1/5], Step [300/1388] Loss: 0.0677
Epoch [1/5], Step [600/1388] Loss: 0.0394
Epoch [1/5], Step [900/1388] Loss: 0.0312
Epoch [1/5], Step [1200/1388] Loss: 0.0360
Epoch [2/5], Step [300/1388] Loss: 0.0047
Epoch [2/5], Step [600/1388] Loss: 0.0099
Epoch [2/5], Step [900/1388] Loss: 0.0141
Epoch [2/5], Step [1200/1388] Loss: 0.0008
Epoch [3/5], Step [300/1388] Loss: 0.0464
Epoch [3/5], Step [600/1388] Loss: 0.0003
Epoch [3/5], Step [900/1388] Loss: 0.0077
Epoch [3/5], Step [1200/1388] Loss: 0.0001
Epoch [4/5], Step [300/1388] Loss: 0.0108
Epoch [4/5], Step [600/1388] Loss: 0.0002
Epoch [4/5], Step [900/1388] Loss: 0.0001
Epoch [4/5], Step [1200/1388] Loss: 0.0002
Epoch [5/5], Step [300/1388] Loss: 0.0011
Epoch [5/5], Step [600/1388] Loss: 0.0000
Epoch [5/5], Step [900/1388] Loss: 0.0018
Epoch [5/5], Step [1200/1388] Loss: 0.0097


In [9]:
# Test the model
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for X, y in test_loader:
        X = X.to(device)
        y = y.to(device).bool()
        output = model(X)
        y_pred = output > 0.5
        total += y.shape[0]
        correct += (y_pred == y).sum().item()

    print('Accuracy of the model on the test images: {:.2f} %'.format(
        100 * correct / total))

Accuracy of the model on the test images: 99.70 %
